<a href="https://colab.research.google.com/github/Sadat-Shakeeb/IPL-Data-Analysis-Using-Python-Libraries/blob/main/Analysis_On_Ball_To_Ball_Datasets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import pandas as pd

## Question on IPL Data

IPL Data Link :  https://drive.google.com/file/d/1xzluAOVoDwpcFg0YiqF0EOUAh__SK2b4/view?usp=drive_link

Download ipl matches 2008-2022 file.

In [14]:
ipl = pd.read_csv('/content/deliveries.csv')
ipl.head()

,match_id,inning,batting_team,bowling_team,over,ball,batsman,non_striker,bowler,is_super_over,...,bye_runs,legbye_runs,noball_runs,penalty_runs,batsman_runs,extra_runs,total_runs,player_dismissed,dismissal_kind,fielder
0,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,1,DA Warner,S Dhawan,TS Mills,0,...,0,0,0,0,0,0,0,NaN,NaN,NaN
1,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,2,DA Warner,S Dhawan,TS Mills,0,...,0,0,0,0,0,0,0,NaN,NaN,NaN
2,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,3,DA Warner,S Dhawan,TS Mills,0,...,0,0,0,0,4,0,4,NaN,NaN,NaN
3,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,4,DA Warner,S Dhawan,TS Mills,0,...,0,0,0,0,0,0,0,NaN,NaN,NaN
4,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,5,DA Warner,S Dhawan,TS Mills,0,...,0,0,0,0,0,2,2,NaN,NaN,NaN


In [4]:
ipl.shape

(179078, 21)

In [5]:
# find the top 10 batsman in terms of runs
ipl.groupby('batsman')['batsman_runs'].sum().sort_values(ascending=False).head(10)

,batsman_runs
batsman,
V Kohli,5434
SK Raina,5415
RG Sharma,4914
DA Warner,4741
S Dhawan,4632
CH Gayle,4560
MS Dhoni,4477
RV Uthappa,4446
AB de Villiers,4428


In [6]:
# find the batsman with max no of sixes
six = ipl[ipl['batsman_runs'] == 6]

six.groupby('batsman')['batsman'].count().sort_values(ascending=False).head(1).index[0]

'CH Gayle'

In [7]:
# find batsman with most number of 4's and 6's in last 5 overs
temp_df = ipl[ipl['over'] > 15]
temp_df = temp_df[(temp_df['batsman_runs'] == 4) | (temp_df['batsman_runs'] == 6)]
temp_df.groupby('batsman')['batsman'].count().sort_values(ascending=False).head(1).index[0]

'MS Dhoni'

In [8]:
# find V Kohli's record against all teams
temp_df = ipl[ipl['batsman'] == 'V Kohli']

temp_df.groupby('bowling_team')['batsman_runs'].sum().reset_index()

,bowling_team,batsman_runs
0,Chennai Super Kings,749
1,Deccan Chargers,306
2,Delhi Capitals,66
3,Delhi Daredevils,763
4,Gujarat Lions,283
5,Kings XI Punjab,636
6,Kochi Tuskers Kerala,50
7,Kolkata Knight Riders,675
8,Mumbai Indians,628
9,Pune Warriors,128


In [15]:
# Create a function that can return the highest score of any batsman
temp_df = ipl[ipl['batsman'] == 'V Kohli']
temp_df.groupby('match_id')['batsman_runs'].sum().sort_values(ascending=False).head(1).values[0]

113

In [16]:
def highest(batsman):
  temp_df = ipl[ipl['batsman'] == batsman]
  return temp_df.groupby('match_id')['batsman_runs'].sum().sort_values(ascending=False).head(1).values[0]

In [17]:
highest('DA Warner')

126

## Below Question on IPL Data

IPL Data Link :  https://docs.google.com/spreadsheets/d/1ROM5oTEEMXfnBHAmz3XC5lwGMgnQaw80PiXadKHRgGU/edit?usp=sharing

Download ipl matches 2008-2022 file.

### `Q:` Find batsman in below category-
* Highest score while chasing
* Best Strike rate while chasing and have faced 100+ balls


> Chasing mean team batting in second inning

In [22]:
balls = pd.read_csv("/content/ipl_deliveries.csv")

In [23]:
#1
df = balls[balls.innings == 2]
df.groupby(["ID", "batter"]).sum().sort_values(by = "batsman_run", ascending = False)["batsman_run"].head()

,,batsman_run
ID,batter,
501206,PC Valthaty,120
501243,V Sehwag,119
1254061,SV Samson,119
1136620,SR Watson,117
336018,ST Jayasuriya,114


In [24]:
#2
temp_df = df[~(df.extra_type == "wides")]
temp_df = temp_df.groupby("batter").agg(
{
    "batsman_run": "sum",
    "ballnumber": "count"
})
temp_df["strike_rate"] = temp_df["batsman_run"] / temp_df["ballnumber"] * 100
temp_df[temp_df["ballnumber"] >= 100].sort_values("strike_rate", ascending = False).reset_index().head()

,batter,batsman_run,ballnumber,strike_rate
0,PJ Cummins,222,114,194.736842
1,AD Russell,986,570,172.982456
2,LS Livingstone,182,107,170.093458
3,SP Narine,599,356,168.258427
4,SO Hetmyer,330,200,165.000000


### `Q` Most Successful bowler against any batsman. Find that pair of bowler and batsman.
> Most Successful in terms of dissmissal. A bowler who have dissmissed any batsman most no of times. If any two pairs have same no of dissmisal, consider runs conceded by bowler to that batsman. Those who have concede lesser runs is more successful.

In [25]:

balls["IsBatterOut"] = (balls.batter == balls.player_out)&(~balls.kind.isin(['run out', 'retired hurt', 'retired out']))
balls.groupby(["bowler", "batter"]).agg({
    "IsBatterOut": "sum",
    "batsman_run": "sum"
}).sort_values(by=["IsBatterOut", "batsman_run"], ascending = [False, True]).head(10)

,,IsBatterOut,batsman_run
bowler,batter,,
Z Khan,MS Dhoni,7,74
Sandeep Sharma,V Kohli,7,78
A Mishra,RG Sharma,7,87
R Ashwin,RV Uthappa,7,123
SP Narine,RG Sharma,7,137
R Vinay Kumar,RG Sharma,6,22
YS Chahal,Q de Kock,6,44
JJ Bumrah,RR Pant,6,48
RA Jadeja,GJ Maxwell,6,49


### `Q`: Most successful batting pair in IPL. Batting pair who have scored most runs playing together.


In [26]:
def func(x):
    return '-'.join(list(np.sort(x.values)))
balls["batter-pair"] = balls[["batter", "non-striker"]].apply(func, axis = 1)
balls.groupby("batter-pair")["total_run"].sum().sort_values(ascending = False).head()

,total_run
batter-pair,
AB de Villiers-V Kohli,3134
CH Gayle-V Kohli,2802
DA Warner-S Dhawan,2357
G Gambhir-RV Uthappa,1906
KL Rahul-MA Agarwal,1731


### `Q-8:` Make a dataframe for all batting pairs played together.
```
Batsman1 Batsman2 Runs Avg StrikeRate
```

> Just to ease this question you can count wide-balls for strike rate.

In [27]:
temp_df = balls.groupby("batter-pair").agg(
{
    "total_run": "sum",
    "ballnumber": "count",
    "isWicketDelivery": "sum"
}).reset_index()
temp_df["Batsman 1"] = temp_df["batter-pair"].apply(lambda x: x.split("-")[0])
temp_df["Batsman 2"] = temp_df["batter-pair"].apply(lambda x: x.split("-")[1])
temp_df.rename(columns = {"total_run": "Runs"}, inplace=True)
temp_df["StrikeRate"] = temp_df["Runs"] / temp_df["ballnumber"] * 100
temp_df["Avg"] = temp_df["Runs"] / temp_df["isWicketDelivery"]
temp_df.sort_values("Runs", ascending = False, inplace=  True)
temp_df[["Batsman 1", "Batsman 2", "Runs", "Avg", "StrikeRate"]]

,Batsman 1,Batsman 2,Runs,Avg,StrikeRate
302,AB de Villiers,V Kohli,3134,44.140845,152.209811
1251,CH Gayle,V Kohli,2802,52.867925,142.017233
1508,DA Warner,S Dhawan,2357,48.102041,136.637681
1954,G Gambhir,RV Uthappa,1906,39.708333,133.754386
2803,KL Rahul,MA Agarwal,1731,52.454545,142.939719
...,...,...,...,...,...
350,AD Mascarenhas,Gurkeerat Singh,0,0.000000,0.000000
3552,NLTC Perera,RV Gomez,0,0.000000,0.000000
2270,Iqbal Abdulla,TS Mills,0,0.000000,0.000000
3556,NM Coulter,Nile,0,0.000000,0.000000


## Question on IPL Data

IPL_Matches :  https://drive.google.com/file/d/1w7DYDW13hfj99S3DGAng1CmTPOlQ5S3Y/view?usp=drive_link


IPL_BALL_BY_BALL Data link: https://drive.google.com/file/d/1NpCPtn7zN24-tuSs-UYdaJjZrCiQV0qN/view?usp=drive_link

Download ipl matches 2008-2022 file.

### `Q` From the IPL above datasets you have to find the Purple cap holder each season.

*Note: Bowler with most no wickets in a season gets purple cap. If more than one bowler have same no of wickets in the season, one with least ecomnomy among them is purple cap holder.*

Bowler's Economy = runs-conceded per six balls

In [28]:
balls = pd.read_csv("IPL_Ball_by_Ball_2008_2022.csv")
matches = pd.read_csv("IPL_Matches_2008_2022.csv")

In [29]:
seasondf = balls.merge(matches[["ID", "Season"]], on="ID")
seasondf["IsBowlerWicket"] = seasondf.kind.apply(lambda x: 1 if x in ["caught", 'caught and bowled', 'bowled', 'stumped','lbw', 'hit wicket'] else 0)
seasondf["BowlerRun"] = seasondf.extra_type.apply(lambda x: 0 if x in ["legbyes", "byes"] else 1) * seasondf["total_run"]
seasondf["IsLegalBall"] = seasondf.extra_type.apply(lambda x: 0 if x in ["wides", "noballs"] else 1)
pcapdf = seasondf.groupby(["Season", "bowler"], as_index = False)[["IsBowlerWicket", "BowlerRun", "IsLegalBall"]].sum()
pcapdf["Economy"] = pcapdf["BowlerRun"] / pcapdf["IsLegalBall"] * 6
pcapdf.sort_values(["IsBowlerWicket", "Economy"], ascending = [False, True]).drop_duplicates("Season", keep = "first").sort_values("Season")

,Season,bowler,IsBowlerWicket,BowlerRun,IsLegalBall,Economy
84,2007/08,Sohail Tanvir,22,266,247,6.461538
174,2009,RP Singh,23,417,358,6.988827
284,2009/10,PP Ojha,21,429,353,7.291785
447,2011,SL Malinga,28,375,378,5.952381
537,2012,M Morkel,25,453,378,7.190476
629,2013,DJ Bravo,32,497,375,7.952000
779,2014,MM Sharma,23,452,323,8.396285
847,2015,DJ Bravo,26,426,314,8.140127
938,2016,B Kumar,23,490,396,7.424242
1048,2017,B Kumar,26,369,314,7.050955


### `Q:` Best bowler in death overs.
*Note: Have taken most no of wickets in case of tie with least economy*

Death Overs - [16-20]

In [30]:
death_overs = seasondf[seasondf.overs>=15]
pcapdf = death_overs.groupby("bowler", as_index = False)[["IsBowlerWicket", "BowlerRun", "IsLegalBall"]].sum()
pcapdf["Economy"] = pcapdf["BowlerRun"] / pcapdf["IsLegalBall"] * 6
pcapdf.sort_values(["IsBowlerWicket", "Economy"], ascending = [False, True]).head()

,bowler,IsBowlerWicket,BowlerRun,IsLegalBall,Economy
91,DJ Bravo,115,2161,1385,9.361733
331,SL Malinga,108,1464,1117,7.863921
53,B Kumar,90,1752,1173,8.961637
148,JJ Bumrah,80,1614,1146,8.450262
339,SP Narine,67,1164,949,7.359326


### `Q` Batsman record season wise

Make a function which takes a input `batsman_name` and it returns a dataframe.
Columns of the data frame are - `['Season','Innings', 'TotalRuns', 'Avg', 'HighestScore','StrikeRate']`.
* In result make `Season` column as index.

* Avg - total_runs/ no of time got out. - player_out column will help.
* StrikeRate -(total_runs/ balls faced) * 100- wides are not included in batsman ball faced counts. No balls are included. -> Extra_type column will help
* Batsman Can score runs on No Balls.
* Batsman can get out on No Ball or Wides. And even while being on non-striker. Keep these things in mind before masking.

In [31]:
batterdf = seasondf.copy()
batterdf["IsBatsmanBall"] = batterdf["extra_type"].apply(lambda x: 1 if x != "wides" else 0)

def bat_record_season(batsman):
    bdf = batterdf[batterdf.batter == batsman].copy()
    bdf["IsBatsmanOut"] = bdf.batter == bdf.player_out
    df = bdf.groupby(["Season", "ID"], as_index = False)[["batsman_run", "IsBatsmanBall", "IsBatsmanOut"]].sum()
    innings = df.groupby("Season").ID.count()
    df = df.groupby("Season").agg(
    {
        "batsman_run":["sum", "max"],
        "IsBatsmanBall": "sum",
        "IsBatsmanOut": "sum"
    })
    df["Innings"]= innings
    df["TotalRuns"] = df[("batsman_run", "sum")]
    df["Avg"] = df["TotalRuns"] / df[("IsBatsmanOut", "sum")]
    df["HighestScore"] = df[("batsman_run", "max")]
    df["StrikeRate"] = df["TotalRuns"] / df[("IsBatsmanBall", "sum")] * 100
    return df.drop(columns = ["batsman_run", "IsBatsmanBall", "IsBatsmanOut"])
bat_record_season("MS Dhoni")

,Innings,TotalRuns,Avg,HighestScore,StrikeRate
,,,,,
Season,,,,,
2007/08,14,414,41.400000,65,133.548387
2009,13,332,41.500000,58,127.203065
2009/10,11,287,31.888889,66,136.666667
2011,13,392,43.555556,70,158.704453
2012,17,358,32.545455,51,128.776978
2013,16,461,46.100000,67,162.897527
2014,15,371,74.200000,57,148.400000
2015,17,372,31.000000,53,121.967213


### `Q` Using both dataset, make a dataframe as described below

Data Frame columns-> `['PlayerOfThematch', 'BattingFigure', 'BowlingFigure']`

* BattingFigure->`<runs>/<balls>`
* BowlingFigure->`<wicket>/<runs-conceded>`

DataFrame should have one record for each match.

Say 'V Kohli' got POM award then in dataset include his batting figure of that match. Say he scored 112runs in 76 balls. And he hasn't bowled so Bowling Figure will be NaN
```
PlayerOfThematch BattingFigure BowlingFigure
V Kohli          112/76         nan  

```


In [32]:
df = balls.merge(matches[["ID", "Player_of_Match"]], on="ID")
batterdf = df[df.batter == df.Player_of_Match].copy()
batterdf["IsBatsmanBall"] = batterdf["extra_type"].apply(lambda x: 1 if x != "wides" else 0)
batter = batterdf.groupby(["ID", "batter"], as_index=False)[["batsman_run", "IsBatsmanBall"]].sum()
batter["BattingFigure"] = batter[["batsman_run", "IsBatsmanBall"]].apply(lambda x: '/'.join(map(str, x.values)), axis = 1)
batter.rename(columns = {"batter": "PlayerOfMatch"}, inplace = True)
batter.head()

,ID,PlayerOfMatch,batsman_run,IsBatsmanBall,BattingFigure
0,335982,BB McCullum,158,73,158/73
1,335983,MEK Hussey,116,54,116/54
2,335985,MV Boucher,39,19,39/19
3,335986,DJ Hussey,38,43,38/43
4,335987,SR Watson,76,49,76/49


In [33]:
bowlerdf = df[df.bowler == df.Player_of_Match].copy()
bowlerdf["IsBowlerWicket"] = bowlerdf["kind"].apply(lambda x: 1 if x in ["caught", 'caught and bowled', 'bowled', 'stumped','lbw', 'hit wicket'] else 0)
bowlerdf["BowlerRun"] = bowlerdf.extra_type.apply(lambda x: 0 if x in ["legbyes", "byes"] else 1) * bowlerdf["total_run"]
bowler = bowlerdf.groupby(["ID", "bowler"], as_index=False)[["IsBowlerWicket", "BowlerRun"]].sum()
bowler["BowlingFigure"] = bowler[["IsBowlerWicket", "BowlerRun"]].apply(lambda x: '/'.join(map(str, x.values)), axis = 1)
bowler.rename(columns = {"bowler": "PlayerOfMatch"}, inplace = True)
bowler.head()

,ID,PlayerOfMatch,IsBowlerWicket,BowlerRun,BowlingFigure
0,335984,MF Maharoof,2,11,2/11
1,335986,DJ Hussey,1,35,1/35
2,335987,SR Watson,0,39,0/39
3,335990,YK Pathan,2,20,2/20
4,335992,SR Watson,2,20,2/20


In [34]:
batter.merge(bowler, on=["ID", "PlayerOfMatch"], how = "outer").drop(columns = ["batsman_run", "IsBatsmanBall", "IsBowlerWicket", "BowlerRun"]).head()

,ID,PlayerOfMatch,BattingFigure,BowlingFigure
0,335982,BB McCullum,158/73,NaN
1,335983,MEK Hussey,116/54,NaN
2,335984,MF Maharoof,NaN,2/11
3,335985,MV Boucher,39/19,NaN
4,335986,DJ Hussey,38/43,1/35
